In [352]:
import pandas as pd
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from random import randint

%matplotlib inline

# Loading Data

In [353]:
df = pd.read_csv('data.csv',index_col=0)
df

,location,country,gender,age,vis_wuhan,from_wuhan,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,diff_sym_hos,result
0,104,8,1,66.0,1,0,14,31,19,12,3,1,8,1
1,101,8,0,56.0,0,1,14,31,19,12,3,1,0,0
2,137,8,1,46.0,0,1,14,31,19,12,3,1,13,0
3,116,8,0,60.0,1,0,14,31,19,12,3,1,0,0
4,116,8,1,58.0,0,0,14,31,19,12,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,48,3,2,24.0,0,0,14,31,19,12,3,1,0,1
859,0,0,2,35.0,0,0,14,31,19,12,3,1,0,0
860,3,1,1,49.4,0,0,14,31,19,12,3,1,0,0
861,24,9,1,49.4,0,0,14,31,19,12,3,1,0,0


# correlation matrix

In [354]:
df.corr()

,location,country,gender,age,vis_wuhan,from_wuhan,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,diff_sym_hos,result
location,1.000000,0.309347,0.030077,-0.027330,0.022821,0.080120,0.033078,0.017961,0.010472,-0.026823,0.033692,0.015341,-0.119397,0.046240
country,0.309347,1.000000,0.193208,0.072339,-0.149798,-0.194170,-0.078971,-0.085210,-0.042933,-0.018526,-0.009371,-0.004384,-0.146037,-0.094443
gender,0.030077,0.193208,1.000000,-0.020662,-0.005588,-0.138997,0.146706,0.116159,0.090858,0.021882,0.026320,0.039867,-0.176528,-0.061441
age,-0.027330,0.072339,-0.020662,1.000000,-0.157173,-0.051229,-0.209028,-0.127251,-0.074863,-0.069396,0.049817,0.055145,0.074573,0.515127
vis_wuhan,0.022821,-0.149798,-0.005588,-0.157173,1.000000,-0.154200,0.087330,0.094360,0.086113,0.063738,0.025780,0.016062,0.065908,-0.151122
from_wuhan,0.080120,-0.194170,-0.138997,-0.051229,-0.154200,1.000000,0.036539,0.046923,0.068144,0.021704,0.018999,0.011837,0.019706,0.128314
symptom1,0.033078,-0.078971,0.146706,-0.209028,0.087330,0.036539,1.000000,0.519404,0.260397,0.095402,-0.002658,0.086430,-0.141160,-0.163039
symptom2,0.017961,-0.085210,0.116159,-0.127251,0.094360,0.046923,0.519404,1.000000,0.530968,0.329583,0.159823,0.072975,-0.119052,-0.103270
symptom3,0.010472,-0.042933,0.090858,-0.074863,0.086113,0.068144,0.260397,0.530968,1.000000,0.429853,0.120301,0.146347,-0.124293,-0.074982
symptom4,-0.026823,-0.018526,0.021882,-0.069396,0.063738,0.021704,0.095402,0.329583,0.429853,1.000000,0.277558,0.110553,-0.089592,-0.108723


In [355]:
test_size = 0.1
validaition_size = 0.15
cv = 10

# unknown gender and negative hours

In [356]:
df['gender'] = df['gender'].mask(df['gender'] == 2, 1)
for i in range(len(df)):
    df.iloc[i, 12] = df.iloc[i,12]*-1 if df.iloc[i, 12] < 0 else df.iloc[i, 12]

# Training, Validation, Testing Functions

In [357]:
def train_validate(X_train, X_validate, y_train, y_validate):
    dic = {'k':[], 'weights':[],'precision':[], 'recall':[], 'F1':[], 'roc_auc':[]}
    weights = ['distance', 'uniform']
    for k in range(1, int(len(X_train)**0.5), 2):
            for j in range(2):
                knn = KNeighborsClassifier(n_neighbors=k, weights=weights[j])
                knn.fit(X_train, y_train)
                y_predict = knn.predict(X_validate)
                precision= precision_score(y_validate, y_predict)
                recall = recall_score(y_validate,y_predict)
                roc_auc = roc_auc_score(y_validate, y_predict)
                F1 = f1_score(y_validate, y_predict)
                dic['weights'].append(weights[j])
                dic['k'].append(k)
                dic['precision'].append(precision)
                dic['recall'].append(recall)
                dic['F1'].append(F1)
                dic['roc_auc'].append(roc_auc)
    return pd.DataFrame(dic)

In [358]:
def train_test(X_train, X_test, y_train, y_test, optimal_k, weights):
    knn = KNeighborsClassifier(n_neighbors=optimal_k, weights=weights)
    knn.fit(X_train, y_train)
    y_predict = knn.predict(X_test)
    precision= precision_score(y_test, y_predict)
    recall = recall_score(y_test,y_predict)
    roc_auc = roc_auc_score(y_test, y_predict)
    F1 = f1_score(y_test, y_predict)
    return precision, recall, F1, roc_auc

# training wihtout one hot encoding

In [359]:
x = df[['location', 'country', 'gender', 'age', 'vis_wuhan', 'from_wuhan',
       'symptom1', 'symptom2', 'symptom3', 'symptom4', 'symptom5', 'symptom6',
       'diff_sym_hos']]
y = df['result']
X_train1, X_test, y_train1, y_test = train_test_split(x, y, test_size=test_size,shuffle=True, random_state=42)
X_train2, X_validate, y_train2, y_validate = train_test_split(X_train1, y_train1, test_size=validaition_size,shuffle=True, random_state=42)

In [360]:
parameters = [
    {'n_neighbors': np.arange(1, int(len(X_train1) ** 0.5), 2)},
]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, parameters, scoring='roc_auc', cv=cv)
grid_search = grid.fit(X_train1, y_train1)
grid_search = grid.fit(X_train1, y_train1)
optimal_k = grid_search.best_params_['n_neighbors']

In [361]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(0.9272727272727272, 0.5204081632653061, 0.6666666666666667, 0.7572542291252785)
predicting on testing
(0.875, 0.7, 0.7777777777777777, 0.8435064935064934)
(1.0, 0.4, 0.5714285714285715, 0.7)


In [362]:
f = train_validate(X_train2, X_validate, y_train2, y_validate)
f[f['roc_auc'] == f['roc_auc'].max()]

,k,weights,precision,recall,F1,roc_auc
11,11,uniform,0.875,0.7,0.777778,0.845327


In [363]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, 11, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, 11, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, 11, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, 11, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(0.8732394366197183, 0.6326530612244898, 0.7337278106508875, 0.809689362470652)
predicting on testing
(0.9, 0.9, 0.9, 0.9435064935064935)
(0.875, 0.7, 0.7777777777777777, 0.8435064935064934)


# encoding
one hot encoding for catagorical data

In [364]:
columns = ['symptom1', 'symptom2', 'symptom3', 'symptom4', 'symptom5', 'symptom6', 'location', 'country']
df_new = df.copy()
for i in columns:
    encoding = pd.get_dummies(df[i], prefix=i)
    df_new = df_new.join(encoding)
    df_new.drop(i, axis=1, inplace=True)
df_new

,gender,age,vis_wuhan,from_wuhan,diff_sym_hos,result,symptom1_0,symptom1_1,symptom1_2,symptom1_3,...,country_24,country_25,country_26,country_27,country_28,country_29,country_30,country_31,country_32,country_33
0,1,66.0,1,0,8,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,56.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,46.0,0,1,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,60.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,58.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,1,24.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
859,1,35.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,1,49.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
861,1,49.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [365]:
x = df_new.drop('result', axis=1)
y = df_new['result']
X_train1, X_test, y_train1, y_test = train_test_split(x, y, test_size=test_size,shuffle=True, random_state=42)
X_train2, X_validate, y_train2, y_validate = train_test_split(X_train1, y_train1, test_size=validaition_size,shuffle=True, random_state=42)

In [366]:
parameters = [
    {'n_neighbors': np.arange(1, int(len(X_train1) ** 0.5), 2)},
]
print(parameters)
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, parameters, scoring='roc_auc', cv=cv)
grid_search = grid.fit(X_train1, y_train1)
optimal_k = grid_search.best_params_['n_neighbors']

[{'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])}]


In [367]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(0.9120879120879121, 0.8469387755102041, 0.8783068783068784, 0.917569682740353)
predicting on testing
(1.0, 0.8, 0.888888888888889, 0.9)
(1.0, 0.7, 0.8235294117647058, 0.85)


In [368]:
f = train_validate(X_train2, X_validate, y_train2, y_validate)
f[f['roc_auc'] == f['roc_auc'].max()]

,k,weights,precision,recall,F1,roc_auc
0,1,distance,0.615385,0.8,0.695652,0.876636
1,1,uniform,0.615385,0.8,0.695652,0.876636


In [369]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, 1, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, 1, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, 1, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, 1, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(1.0, 1.0, 1.0, 1.0)
predicting on testing
(1.0, 0.9, 0.9473684210526316, 0.95)
(1.0, 0.9, 0.9473684210526316, 0.95)


# Normalization
normalization for age
every time i try normalizing the dff_sym_hos it gives me a warning during training

In [370]:
scaler = MinMaxScaler()
df_new['age'] = scaler.fit_transform(df_new['age'].to_numpy()[:,np.newaxis])[:,0]
# df_new['diff_sym_hos'] = scaler.fit_transform(df_new['diff_sym_hos'].to_numpy()[:,np.newaxis])[:,0]
df_new

,gender,age,vis_wuhan,from_wuhan,diff_sym_hos,result,symptom1_0,symptom1_1,symptom1_2,symptom1_3,...,country_24,country_25,country_26,country_27,country_28,country_29,country_30,country_31,country_32,country_33
0,1,0.680851,1,0,8,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.574468,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.468085,0,1,13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.617021,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0.595745,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,1,0.234043,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
859,1,0.351064,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,1,0.504255,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
861,1,0.504255,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [371]:
x = df_new.drop('result', axis=1)
y = df_new['result']
X_train1, X_test, y_train1, y_test = train_test_split(x, y, test_size=test_size,shuffle=True, random_state=42)
X_train2, X_validate, y_train2, y_validate = train_test_split(X_train1, y_train1, test_size=validaition_size,shuffle=True, random_state=42)

In [372]:
parameters = [
    {'n_neighbors': np.arange(1, int(len(X_train1) ** 0.5), 2)},
]
print(parameters)
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, parameters, scoring='roc_auc', cv=cv)
grid_search = grid.fit(X_train1, y_train1)
grid_search = grid.fit(X_train1, y_train1)
optimal_k = grid_search.best_params_['n_neighbors']

[{'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])}]


In [373]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, optimal_k, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, optimal_k, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(0.86, 0.4387755102040816, 0.581081081081081, 0.7142255132141352)
predicting on testing
(0.625, 0.5, 0.5555555555555556, 0.7305194805194805)
(0.6, 0.3, 0.4, 0.6370129870129869)


In [374]:
f = train_validate(X_train2, X_validate, y_train2, y_validate)
f[f['roc_auc'] == f['roc_auc'].max()]

,k,weights,precision,recall,F1,roc_auc
2,3,distance,0.4375,0.7,0.538462,0.807944
3,3,uniform,0.4375,0.7,0.538462,0.807944


In [375]:
print("predicting on training")
print(train_test(X_train1, X_train1, y_train1, y_train1, 3, 'distance'))
print(train_test(X_train1, X_train1, y_train1, y_train1, 3, 'uniform'))
print("predicting on testing")
print(train_test(X_train1, X_test, y_train1, y_test, 3, 'distance'))
print(train_test(X_train1, X_test, y_train1, y_test, 3, 'uniform'))

predicting on training
(1.0, 1.0, 1.0, 1.0)
(0.8809523809523809, 0.7551020408163265, 0.8131868131868131, 0.8701763891397266)
predicting on testing
(0.6363636363636364, 0.7, 0.6666666666666666, 0.824025974025974)
(0.6666666666666666, 0.6, 0.631578947368421, 0.7805194805194805)
